In [ ]:
%conda install -c conda-forge langchain openai python-dotenv -y
%pip install "langserve[all]"

## 1. LLM / ChatModel

가장 기반이 되는 실행 예시

In [1]:
from langchain.schema import HumanMessage
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI

from dotenv import load_dotenv
load_dotenv()

# OpenAI API Key는 자동 import 하므로, load_dotenv()만 잘해주면 됨.

llm = OpenAI()
chat_model = ChatOpenAI()

text = "What would be a good company name for a company that makes colorful socks?"
messages = [HumanMessage(content=text)]

llm_res = llm.invoke(text)

chat_res = chat_model.invoke(messages)

print(f"LLM Response: {llm_res}")
print(f"ChatModel Response: {chat_res}")


LLM Response: 

Socktopia Colors
ChatModel Response: content='Colorful Soles'


## 2. Prompt template

Prompt Template를 활용하는 가장 기본적인 예시

In [2]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template("What is a good name for a company that makes {product}?")
result = prompt.format(product="colorful socks")

print(result)

What is a good name for a company that makes colorful socks?


In [3]:
from langchain.prompts.chat import ChatPromptTemplate

system_template = "You are a helpful assistant that translates {input_language} to {output_language}."
human_template = "{text}"

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", system_template),
    ("human", human_template),
])

messages = chat_prompt.format_messages(input_language="English", output_language="French", text="I love programming.")

print(messages)

[SystemMessage(content='You are a helpful assistant that translates English to French.'), HumanMessage(content='I love programming.')]


## 3. Output parsers

Output Parsers를 활용하는 가장 기본적인 예시

In [4]:
from langchain.schema import BaseOutputParser
from typing import List

class CommaSeparatedListOutputParser(BaseOutputParser):
    def parse(self, text: str) -> List[str]:
        return text.strip().split(", ")

result = CommaSeparatedListOutputParser().parse("hi, bye")

print(result)

['hi', 'bye']


## 4. Composing with LCEL

LCEL을 활용하여 앞 선 module들을 총합하는 예시

In [5]:
from typing import List

from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema import BaseOutputParser

class CommaSeparatedListOutputParser(BaseOutputParser[List[str]]):
    def parse(self, text: str) -> List[str]:
        return text.strip().split(", ")
    
system_template = """You are a helpful assistant who generates comma separated lists.
A user will pass in a category, and you should generate 5 objects in that category in a comma separated list.
ONLY return a comma separated list, and noting more."""
human_template = "{text}"

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", system_template),
    ("human", human_template),
])
chain = chat_prompt | ChatOpenAI() | CommaSeparatedListOutputParser()
result = chain.invoke({"text": "colors"})

print(result)

['red', 'blue', 'green', 'yellow', 'orange']
